In [29]:
!wget "https://firebasestorage.googleapis.com/v0/b/scrapper-6e7db.appspot.com/o/spam.csv?alt=media&token=427d4a71-d22b-4484-ac22-6eb6b7b59c4f" -O "spam.csv"

--2023-08-23 18:42:46--  https://firebasestorage.googleapis.com/v0/b/scrapper-6e7db.appspot.com/o/spam.csv?alt=media&token=427d4a71-d22b-4484-ac22-6eb6b7b59c4f
Resolving firebasestorage.googleapis.com (firebasestorage.googleapis.com)... 74.125.137.95, 142.251.2.95, 2607:f8b0:4023:c0b::5f, ...
Connecting to firebasestorage.googleapis.com (firebasestorage.googleapis.com)|74.125.137.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503663 (492K) [text/csv]
Saving to: ‘spam.csv’

spam.csv            100%[===================>] 491.86K  --.-KB/s    in 0.004s  

2023-08-23 18:42:47 (125 MB/s) - ‘spam.csv’ saved [503663/503663]



In [30]:
import pandas as pd

csv_path = 'spam.csv'

encodings = ['utf-8', 'ISO-8859-1', 'latin1']
for encoding in encodings:
    try:
        df = pd.read_csv(csv_path, encoding=encoding)
        print("CSV file successfully read with encoding:", encoding)
        break
    except UnicodeDecodeError:
        print("Error reading with encoding:", encoding)

print(df.head())

Error reading with encoding: utf-8
CSV file successfully read with encoding: ISO-8859-1
     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


In [31]:
selected_columns = df.iloc[:, :2]
selected_columns.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [32]:
ham_count = (selected_columns['v1'] == 'ham').sum()
spam_count = (selected_columns['v1'] == 'spam').sum()

print("Number of 'ham' entries:", ham_count)
print("Number of 'spam' entries:", spam_count)

Number of 'ham' entries: 4825
Number of 'spam' entries: 747


In [33]:
selected_columns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [34]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

selected_columns['v2'] = selected_columns['v2'].apply(word_tokenize)

stop_words = set(stopwords.words('english'))
selected_columns['v2'] = selected_columns['v2'].apply(lambda tokens: [word for word in tokens if word.lower() not in stop_words])

selected_columns.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-34-bb81ea00bbb5>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-34-bb81ea00bbb5>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,v1,v2
0,ham,"[Go, jurong, point, ,, crazy, .., Available, b..."
1,ham,"[Ok, lar, ..., Joking, wif, u, oni, ...]"
2,spam,"[Free, entry, 2, wkly, comp, win, FA, Cup, fin..."
3,ham,"[U, dun, say, early, hor, ..., U, c, already, ..."
4,ham,"[Nah, n't, think, goes, usf, ,, lives, around,..."


In [35]:
import plotly.express as px

selected_columns['length'] = selected_columns['v2'].apply(len)

length_count_df = selected_columns.groupby(['v1', 'length']).size().reset_index(name='count')
fig = px.area(length_count_df, x='length', y='count', color='v1', title='Tokenized String Length Distribution',
              labels={'length': 'Tokenized String Length', 'count': 'Frequency'},
              color_discrete_map={'ham': 'rgba(0, 0, 255, 0.5)', 'spam': 'rgba(255, 0, 0, 0.5)'})


fig.show()


<ipython-input-35-22cba5716fa4>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

selected_columns['v2'] = selected_columns['v2'].apply(' '.join)
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(selected_columns['v2'])

tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

tfidf_df['v1'] = selected_columns['v1']

<ipython-input-36-f5299b3b42dc>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [37]:
tfidf_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Columns: 8607 entries, 00 to v1
dtypes: float64(8606), object(1)
memory usage: 365.9+ MB


In [42]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_curve, auc, precision_recall_curve
import plotly.graph_objects as go


X = tfidf_df.drop(columns=['v1'])
y = tfidf_df['v1']

y = y.apply(lambda label: 1 if label == 'spam' else 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

logreg_model = LogisticRegression()
logreg_model.fit(X_train, y_train)

# make predictions on the test set
y_pred = logreg_model.predict(X_test)


print("Classification Report for Test Set:")
print(classification_report(y_test, y_pred))

# calculate ROC curve and AUC
fpr, tpr, thresholds = roc_curve(y_test, logreg_model.predict_proba(X_test)[:, 1])
roc_auc = auc(fpr, tpr)

# create ROC curve plot using Plotly
roc_curve_fig = go.Figure()
roc_curve_fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'ROC Curve (AUC = {roc_auc:.2f})'))
roc_curve_fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='random classifier'))
roc_curve_fig.update_layout(title='ROC Curve', xaxis_title='false positive rate', yaxis_title='true positive rate')
roc_curve_fig.show()

# calculate precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_test, logreg_model.predict_proba(X_test)[:, 1])

# create Precision-Recall curve plot
precision_recall_fig = go.Figure()
precision_recall_fig.add_trace(go.Scatter(x=recall, y=precision, mode='lines', name='Precision-Recall Curve'))
precision_recall_fig.update_layout(title='Precision-Recall Curve', xaxis_title='Recall', yaxis_title='Precision')
precision_recall_fig.show()


Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       965
           1       0.95      0.63      0.76       150

    accuracy                           0.95      1115
   macro avg       0.95      0.81      0.86      1115
weighted avg       0.95      0.95      0.94      1115



In [44]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.95


In [47]:
import joblib
import warnings

model_filename = 'spam_detection_model.joblib'
joblib.dump(logreg_model, model_filename)
print(f"Model saved as {model_filename}")


Model saved as spam_detection_model.joblib


In [49]:
example_strings = [
    "Congratulations, you've won a free vacation! Reply now to claim your prize.",
    "Hi, how are you doing?"
]


loaded_model = joblib.load(model_filename)


predictions = []
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    for example_string in example_strings:
        example_tokens = word_tokenize(example_string)
        example_tokens = [word for word in example_tokens if word.lower() not in stop_words]
        example_text = ' '.join(example_tokens)

        example_tfidf = tfidf_vectorizer.transform([example_text])
        prediction = loaded_model.predict(example_tfidf)

        if prediction[0] == 1:
            predictions.append("Spam")
        else:
            predictions.append("Ham")

for i, prediction in enumerate(predictions):
    print(f"Prediction for example {i+1}: {prediction}")

Prediction for example 1: Spam
Prediction for example 2: Ham
